In [1]:
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import jaccard_score, multilabel_confusion_matrix
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn import preprocessing
from skopt import BayesSearchCV
from sklearn.base import BaseEstimator, ClassifierMixin
import librosa
import librosa.display
import librosa.feature
import IPython.display as ipd

In [2]:
x = np.load('../Part1.npy')
Y_t = np.load('../train.npy/y_train.npy')
#X_train = (x-np.min(x))/(np.max(x)-np.min(x))
xt = np.load('../Part2.npy')
Y_test = np.load('../test.npy/Y_Test.npy')
#X_test = (xt-np.min(xt))/(np.max(xt)-np.min(xt))
x_add = np.load('../Part3.npy')
y_add = np.load('../additional.npy/Y_additional.npy')
#x_add = (x_add-np.min(x_add))/(np.max(x_add)-np.min(x_add))
Y_train = np.concatenate((Y_t, Y_test, y_add))
X_train = np.concatenate((x, xt, x_add))
#X_train = (X_train-np.min(X_train))/(np.max(X_train)-np.min(X_train))
scaler = preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

In [3]:
X_train.shape

(2874, 17)

In [4]:
#IRMAS-TrainingData/cel/[cel][pop_roc]0143__3.wav
instruments = ['cel/','cla/','flu/','gac/','gel/','org/','pia/','sax/','tru/','vio/','voi/']
pathAudio = "../IRMAS-TrainingData/"
X_test = []
Y_test = []
cnt = 0
for i in instruments:
    label = np.zeros(11)
    label[cnt] = 1
    cnt +=1
    #print(pathAudio + i)
    files = librosa.util.find_files(pathAudio + i, ext=['wav'], recurse=False) 
    files = np.asarray(files)
    #print(files)
    for y in files:
        #print(test)
        y, sr = librosa.load(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        rolloff = np.mean(rolloff)
        bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        bw = np.mean(bw)
        ctr = librosa.feature.spectral_centroid(y=y, sr=sr)
        ctr = np.mean(ctr)
        zcr = librosa.feature.zero_crossing_rate(y=y)
        zcr = np.mean(zcr)
        mfcc = np.mean(mfcc, axis=1)
        mfcc = mfcc[:13]
        mfcc = np.append(mfcc, [rolloff, bw, ctr, zcr])
        X_test.append(mfcc)
        Y_test.append(label)

In [4]:
X_test = np.load('X_single.npy')
Y_test = np.load('Y_single.npy')

In [5]:
np.array(X_test).shape

(6705, 17)

In [6]:
mlp = MLPClassifier(activation = 'identity',hidden_layer_sizes = [100] ,solver = 'lbfgs',alpha = 0.00003, batch_size = 50,
                    learning_rate_init=0.058, max_iter = 114, warm_start=True)
chains = [ClassifierChain(mlp, order="random", random_state=i) for i in range(10)]
for chain in chains:
    chain.fit(X_train, Y_train)

In [7]:
Y_pred_chains = np.array([chain.predict(X_test) for chain in chains])
chain_jaccard_scores = [
    jaccard_score(Y_test, Y_pred_chain >= 0.5, average="samples")
    for Y_pred_chain in Y_pred_chains
]

Y_pred_ensemble = Y_pred_chains.mean(axis=0)

In [8]:
chain_jaccard_scores

[0.0802258442526899,
 0.08081282624906787,
 0.07923014097510743,
 0.08001491424310216,
 0.07849650225489152,
 0.07970917225950784,
 0.07631902276197579,
 0.08005362025496256,
 0.07976776392883776,
 0.08134228187919465]

In [9]:
Y_pred = np.where(Y_pred_ensemble>0.5,1,0)

In [10]:
Y_pred

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [11]:
m = multilabel_confusion_matrix(Y_test, Y_pred)

In [12]:
m

array([[[ 595, 5722],
        [  70,  318]],

       [[ 289, 5911],
        [   8,  497]],

       [[  99, 6155],
        [  11,  440]],

       [[6068,    0],
        [ 637,    0]],

       [[5891,   54],
        [ 746,   14]],

       [[   0, 6023],
        [   0,  682]],

       [[   0, 5984],
        [   0,  721]],

       [[6079,    0],
        [ 626,    0]],

       [[6128,    0],
        [ 577,    0]],

       [[6125,    0],
        [ 580,    0]],

       [[5263,  664],
        [ 775,    3]]], dtype=int64)

In [13]:
precision = [x[1,1]/(x[1,1]+x[0,1]) for x in m]

C:\Users\jonat\AppData\Local\Temp\ipykernel_22612\2312814414.py:1: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = [x[1,1]/(x[1,1]+x[0,1]) for x in m]


In [14]:
precision

[0.05264900662251656,
 0.07755930087390761,
 0.0667172100075815,
 nan,
 0.20588235294117646,
 0.10171513795674869,
 0.1075316927665921,
 nan,
 nan,
 nan,
 0.004497751124437781]

In [15]:
recall = np.array([x[1,1]/(x[1,1]+x[1,0]) for x in m])


In [16]:
recall

array([0.81958763, 0.98415842, 0.97560976, 0.        , 0.01842105,
       1.        , 1.        , 0.        , 0.        , 0.        ,
       0.00385604])

In [17]:
f1 = 2*(precision*recall/(precision+recall))
f1

array([0.09894213, 0.14378707, 0.12489356,        nan, 0.03381643,
       0.18464871, 0.1941826 ,        nan,        nan,        nan,
       0.00415225])

In [18]:
ovr = OneVsRestClassifier(mlp)
ovr.fit(X_train, Y_train)

OneVsRestClassifier(estimator=MLPClassifier(activation='identity', alpha=3e-05,
                                            batch_size=50,
                                            hidden_layer_sizes=[100],
                                            learning_rate_init=0.058,
                                            max_iter=114, solver='lbfgs',
                                            warm_start=True))

In [19]:
ovr_pred = ovr.predict(X_test)
M_ovr = multilabel_confusion_matrix(Y_test, ovr_pred)
M_ovr

array([[[ 795, 5522],
        [  86,  302]],

       [[ 164, 6036],
        [   4,  501]],

       [[  92, 6162],
        [  11,  440]],

       [[6068,    0],
        [ 637,    0]],

       [[5658,  287],
        [ 733,   27]],

       [[   0, 6023],
        [   0,  682]],

       [[   0, 5984],
        [   0,  721]],

       [[6018,   61],
        [ 625,    1]],

       [[4641, 1487],
        [ 494,   83]],

       [[6125,    0],
        [ 580,    0]],

       [[5314,  613],
        [ 776,    2]]], dtype=int64)